In [37]:
import sys
# first time running you may need to uncomment the install line below
# !{sys.executable} -m pip install yfinance
import pandas as pd
import numpy as np
#!{sys.executable} -m pip install pandas_datareader
#import pandas_datareader as pdr
import yfinance as yf

# change `AX` to whatever stock you want more information on
lookup = "MEDP"

In [38]:
tick = yf.Ticker(lookup)

e = tick.earnings
yoy_metrics = {}
# print(tick.earnings["Revenue"], tick.earnings["Earnings"])


for date, earnings, in e["Earnings"].items():
    yoy_metrics[str(date)] = {"Earnings": earnings}

for date, rev in e["Revenue"].items():
    yoy_metrics[str(date)]["Revenue"] = rev


for title, data in tick.cash_flow.iterrows():
    if title.find("Operating Cash Flow") != -1:
        
        for date, amt in data.items():
            yoy_metrics[str(date.year)]["Cash Flow"] = amt
   

# Display the shizzing
#for title, data in tick.cash_flow.iterrows():
#    print(title)

print(f"Stock: {lookup} (Amounts in thousands)")
print("\tEarnings\tRevenue\t\tOperating Cash Flow")

for y in yoy_metrics.keys():
    pme = f"""{y}\t${yoy_metrics[y]["Earnings"]//1000:,} T\t${yoy_metrics[y]["Revenue"]//1000:,} T"""
    if "Cash Flow" in yoy_metrics[y]: pme += f"""\t${yoy_metrics[y]["Cash Flow"]//1000:,} T"""
    print(pme)

    
guidelines = """\nGUIDELINES FOR INVESTMENT CLUB:
MARKET CAP WITHIN 1 - 10 B RANGE, and 
15 % GROWTH EACH YEAR IN: 
    - EARNINGS
    - REVENUE
    - OPERATING CASH FLOW
"""
print(guidelines)

calculations = {"Average": {"Revenue": [], "Earnings":[], "Cash Flow":[]}}
for y in list(yoy_metrics.keys())[1:]:
    prior = str(int(y) - 1)
    diff_key = f"{prior} - {y}"
    earn_diff = (yoy_metrics[y]["Earnings"] - yoy_metrics[prior]["Earnings"]) / abs(yoy_metrics[prior]["Earnings"]) * 100
    rev_diff = abs((yoy_metrics[y]["Revenue"] - yoy_metrics[prior]["Revenue"]) / yoy_metrics[prior]["Revenue"] * 100)

    calculations[diff_key] = {"Earnings":earn_diff, "Revenue":rev_diff } # "Cash Flow":opcash_diff}
    if "Cash Flow" in yoy_metrics[y] and "Cash Flow" in yoy_metrics[prior]:
        opcash_diff = (yoy_metrics[y]["Cash Flow"] - yoy_metrics[prior]["Cash Flow"]) / abs(yoy_metrics[prior]["Cash Flow"]) * 100
        calculations[diff_key]["Cash Flow"] = opcash_diff
        calculations["Average"]["Cash Flow"].append(opcash_diff)

    
    calculations["Average"]["Revenue"].append(rev_diff)
    calculations["Average"]["Earnings"].append(earn_diff)
    

#print(tick.info)
print(f"Stock: {tick.info['longName']} Market Cap: {tick.fast_info['market_cap']/ 1_000_000_000:.3f} B")
print("Year\t\tEarnings\tRevenue\t\tOperating Cash Flow")

for y in calculations.keys():
    if y == "Average" : continue
    pme = f"""{y}\t{calculations[y]['Earnings']:.2f}%"""
    pme += f"\t\t{calculations[y]['Revenue']:.2f}%"
    if "Cash Flow" in calculations[y]:
        pme += f"\t\t{calculations[y]['Cash Flow']:.2f}%"
    print(pme)

av_calc = f"Average\t\t{np.mean(calculations['Average']['Earnings']):.2f}%"
av_calc += f"\t\t{np.mean(calculations['Average']['Revenue']):.2f}%"
av_calc += f"\t\t{np.mean(calculations['Average']['Cash Flow']):.2f}%"
print(av_calc)

Stock: MEDP (Amounts in thousands)
	Earnings	Revenue		Operating Cash Flow
2018	$73,185 T	$704,589 T
2019	$100,443 T	$860,969 T	$201,867.0 T
2020	$145,384 T	$925,925 T	$258,676.0 T
2021	$181,848 T	$1,142,377 T	$263,327.0 T

GUIDELINES FOR INVESTMENT CLUB:
MARKET CAP WITHIN 1 - 10 B RANGE, and 
15 % GROWTH EACH YEAR IN: 
    - EARNINGS
    - REVENUE
    - OPERATING CASH FLOW

Stock: Medpace Holdings, Inc. Market Cap: 7.416 B
Year		Earnings	Revenue		Operating Cash Flow
2018 - 2019	37.25%		22.19%
2019 - 2020	44.74%		7.54%		28.14%
2020 - 2021	25.08%		23.38%		1.80%
Average		35.69%		17.71%		14.97%
